In [39]:
# Imports and Database Connection
import sqlite3
import pandas as pd
import requests
from sqlalchemy import create_engine
from IPython.display import display


In [40]:
# Connect to SQLite DB
db_path = "cricbuzz.db"
engine = create_engine(f"sqlite:///{db_path}")
conn = sqlite3.connect("cricbuzz.db")
print("Connected to SQLite Database")

Connected to SQLite Database


In [41]:
# Fetch recent matches data from Cricbuzz API
url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
headers = {
    "x-rapidapi-key": "5670032a90mshae1f46da8637796p14d1ccjsnf3b7630272dd",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

def fetch_recent_matches():
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        matches = []
        for match_type in data.get("typeMatches", []):
            for series in match_type.get("seriesMatches", []):
                ad_wrapper = series.get("seriesAdWrapper", {})
                series_name = ad_wrapper.get("seriesName", "Unknown Series")
                for match in ad_wrapper.get("matches", []):
                    info = match.get("matchInfo", {})
                    team1 = info.get("team1", {}).get("teamName", "")
                    team2 = info.get("team2", {}).get("teamName", "")
                    status = info.get("status", "")
                    start_time = info.get("startDate", "")
                    venue = info.get("venueInfo", {}).get("ground", "") if info.get("venueInfo") else ""
                    matches.append({
                        "Series": series_name,
                        "Team 1": team1,
                        "Team 2": team2,
                        "Status": status,
                        "Start Time": start_time,
                        "Venue": venue
                    })
        df = pd.DataFrame(matches)
        print(f"Fetched {len(df)} recent matches via API.")
        return df
    else:
        print("Error fetching API data:", response.status_code)
        return pd.DataFrame()

recent_matches_df = fetch_recent_matches()
display(recent_matches_df)


Fetched 61 recent matches via API.


,Series,Team 1,Team 2,Status,Start Time,Venue
0,Asia Cup 2025,Bangladesh,Sri Lanka,Sri Lanka won by 6 wkts,1757773800000,Sheikh Zayed Stadium
1,Asia Cup 2025,Afghanistan,Hong Kong,Afghanistan won by 94 runs,1757428200000,Sheikh Zayed Stadium
2,Asia Cup 2025,Hong Kong,Bangladesh,Bangladesh won by 7 wkts,1757601000000,Sheikh Zayed Stadium
3,Asia Cup 2025,Pakistan,Oman,Pakistan won by 93 runs,1757687400000,Dubai International Cricket Stadium
4,Asia Cup 2025,United Arab Emirates,India,India won by 9 wkts,1757514600000,Dubai International Cricket Stadium
...,...,...,...,...,...,...
56,"Luxembourg Womens T20I Tri-Series, 2025",Austria Women,Luxembourg Women,Austria Women won by 54 runs (DLS method) - Re...,1757752200000,Pierre Werner Cricket Ground
57,"Luxembourg Womens T20I Tri-Series, 2025",Switzerland Women,Luxembourg Women,Switzerland Women won by 74 runs,1757685600000,Pierre Werner Cricket Ground
58,Womens Caribbean Premier League 2025,Trinbago Knight Riders Women,Guyana Amazon Warriors Women,Guyana Amazon Warriors Women won by 5 wkts,1757793600000,Providence Stadium
59,Womens Caribbean Premier League 2025,Barbados Royals Women,Trinbago Knight Riders Women,Barbados Royals Women won by 59 runs,1757527200000,Providence Stadium


### SQL Queries

In [42]:
# Query 1: Find all players who represent India
query1 = """
SELECT full_name, playing_role, batting_style, bowling_style
FROM players
WHERE country = 'India';
"""
df1 = pd.read_sql(query1, conn)
print("Query 1: Players representing India")
display(df1)


Query 1: Players representing India


,full_name,playing_role,batting_style,bowling_style
0,Rohit Sharma,Batsman,Right-hand,Right-arm offbreak
1,Virat Kohli,Batsman,Right-hand,Right-arm medium
2,Jasprit Bumrah,Bowler,Right-hand,Right-arm fast


In [43]:
# Query 2: Custom player stats
query2 = """
SELECT p.full_name, s.format, s.runs, s.wickets
FROM player_format_summary s
JOIN players p ON p.player_id = s.player_id
WHERE s.format = 'ODI'
ORDER BY s.runs DESC
LIMIT 5;
"""
df2 = pd.read_sql(query2, conn)
print("Sample Query 2: Top ODI Batters")
display(df2)

Sample Query 2: Top ODI Batters


,full_name,format,runs,wickets
0,Virat Kohli,ODI,13800,4
1,Rohit Sharma,ODI,10700,8
2,Jos Buttler,ODI,5200,0
3,Glenn Maxwell,ODI,3800,50
4,Pat Cummins,ODI,450,190


In [44]:
# Query 3: Venues with > 50k capacity
query3 = """
SELECT venue_name, city, country, capacity
FROM venues
WHERE capacity > 50000
ORDER BY capacity DESC;
"""
df3 = pd.read_sql(query3, conn)
print("Query 3: Venues with >50k capacity")
display(df3)

Query 3: Venues with >50k capacity


,venue_name,city,country,capacity
0,MCG,Melbourne,Australia,100024
1,Eden Gardens,Kolkata,India,68000


In [45]:
# Query 4: Count of matches won by each team
query4 = """
SELECT t.team_name, COUNT(*) AS wins
FROM matches m
JOIN teams t ON m.winner_team_id = t.team_id
GROUP BY t.team_id
ORDER BY wins DESC;
"""
df4 = pd.read_sql(query4, conn)
print("Query 4: Matches won by each team")
display(df4)

Query 4: Matches won by each team


,team_name,wins
0,Australia,1
1,India,1


In [46]:
# Query 5: Count of players by playing role
query5 = """
SELECT playing_role, COUNT(*) AS player_count
FROM players
GROUP BY playing_role
ORDER BY player_count DESC;
"""
df5 = pd.read_sql(query5, conn)
print("Query 5: Players by role")
display(df5)


Query 5: Players by role


,playing_role,player_count
0,Bowler,2
1,Batsman,2
2,Wicket-keeper,1
3,All-rounder,1


In [47]:
# Query 6: Highest individual batting score per format
query6 = """
SELECT format, MAX(runs) AS highest_runs
FROM player_format_summary
GROUP BY format;
"""
df6 = pd.read_sql(query6, conn)
print("Query 6: Highest total runs per cricket format")
display(df6)

Query 6: Highest total runs per cricket format


,format,highest_runs
0,ODI,13800


In [48]:
# Query 7:  Cricket Series started in 2024 with details
query7 = """
SELECT series_name, host_country, match_type, start_date, total_matches
FROM series
WHERE strftime('%Y', start_date) = '2024';
"""
df7 = pd.read_sql(query7, conn)
print("Query 7: Series started in 2024")
display(df7)

Query 7: Series started in 2024


,series_name,host_country,match_type,start_date,total_matches
0,Border-Gavaskar Trophy,India,Test,2024-02-01,5
1,ODI Series IND vs AUS,India,ODI,2024-09-10,3


##### Intermediate Queries

In [49]:
# Query 8: Last 20 completed matches with details
query8 = """
SELECT m.match_desc, t1.team_name AS team1, t2.team_name AS team2,
       w.team_name AS winning_team, m.victory_margin, m.victory_type, v.venue_name
FROM matches m
JOIN teams t1 ON m.team1_id = t1.team_id
JOIN teams t2 ON m.team2_id = t2.team_id
JOIN teams w ON m.winner_team_id = w.team_id
JOIN venues v ON m.venue_id = v.venue_id
ORDER BY m.match_date DESC
LIMIT 20;
"""
df8 = pd.read_sql(query8, conn)
print("Query 8: Last 20 matches details")
display(df8)

Query 8: Last 20 matches details


,match_desc,team1,team2,winning_team,victory_margin,victory_type,venue_name
0,India vs Australia 2nd ODI,India,Australia,Australia,3,wickets,Eden Gardens
1,India vs Australia 1st ODI,India,Australia,India,45,runs,Wankhede Stadium


In [50]:
# Query 9: Player performance comparison across formats
query9 = """
WITH batting_stats AS (
    SELECT
        player_id,
        COALESCE(SUM(CASE WHEN format = 'Test' THEN runs ELSE 0 END), 0) AS test_runs,
        COALESCE(SUM(CASE WHEN format = 'ODI' THEN runs ELSE 0 END), 0) AS odi_runs,
        COALESCE(SUM(CASE WHEN format = 'T20I' THEN runs ELSE 0 END), 0) AS t20i_runs,
        AVG(batting_average) AS overall_avg,
        COUNT(DISTINCT format) AS formats_played
    FROM player_format_summary
    GROUP BY player_id
)
SELECT 
    p.full_name,
    b.test_runs,
    b.odi_runs,
    b.t20i_runs,
    ROUND(b.overall_avg, 2) AS overall_avg,
    b.formats_played
FROM batting_stats b
JOIN players p ON p.player_id = b.player_id
WHERE b.formats_played >= 1
ORDER BY (b.test_runs + b.odi_runs + b.t20i_runs) DESC
LIMIT 50;
"""

df9_mod = pd.read_sql(query9, conn)
print("Query 9: Player performance across formats with at least 1 format")
display(df9_mod)

Query 9: Player performance across formats with at least 1 format


,full_name,test_runs,odi_runs,t20i_runs,overall_avg,formats_played
0,Virat Kohli,0,13800,0,57.7,1
1,Rohit Sharma,0,10700,0,48.5,1
2,Jos Buttler,0,5200,0,41.0,1
3,Glenn Maxwell,0,3800,0,35.8,1
4,Pat Cummins,0,450,0,NaN,1
5,Jasprit Bumrah,0,60,0,NaN,1


In [51]:
# Query 10: Team performance home vs away
query10 = """
SELECT t.team_name,
       SUM(CASE WHEN v.country = t.country AND m.winner_team_id = t.team_id THEN 1 ELSE 0 END) AS home_wins,
       SUM(CASE WHEN v.country != t.country AND m.winner_team_id = t.team_id THEN 1 ELSE 0 END) AS away_wins
FROM matches m
JOIN teams t ON t.team_id IN (m.team1_id, m.team2_id)
JOIN venues v ON m.venue_id = v.venue_id
GROUP BY t.team_id;
"""
df10 = pd.read_sql(query10, conn)
print("Query 10: Team performance home vs away")
display(df10)

Query 10: Team performance home vs away


,team_name,home_wins,away_wins
0,India,1,0
1,Australia,0,1


In [52]:
# Query 11: Players scoring 50 or more runs in an innings
query11 = """
SELECT p.full_name, bs.innings_id, bs.runs
FROM batting_scorecard bs
JOIN players p ON bs.player_id = p.player_id
WHERE bs.runs >= 50
ORDER BY bs.runs DESC;
"""

df11 = pd.read_sql(query11, conn)
print("Query 11: Players scoring 50 or more runs in an innings:")
display(df11)

Query 11: Players scoring 50 or more runs in an innings:


,full_name,innings_id,runs
0,Virat Kohli,1,95
1,Rohit Sharma,1,80
2,Glenn Maxwell,1,75
3,Jos Buttler,2,65
4,Pat Cummins,2,60


In [53]:
# Query 12: Player performance in close matches (without matches won)
query12 = """
SELECT p.full_name,
       AVG(bs.runs) AS avg_runs_in_close,
       COUNT(DISTINCT bs.match_id) AS close_matches_played
FROM batting_scorecard bs
JOIN players p ON bs.player_id = p.player_id
JOIN matches m ON bs.match_id = m.match_id
WHERE ((m.victory_type = 'runs' AND m.victory_margin < 50)
       OR (m.victory_type = 'wickets' AND m.victory_margin < 5))
GROUP BY p.player_id;
"""

df12 = pd.read_sql(query12, conn)
print("Query 12: Player performance in close matches (without matches won)")
display(df12)

Query 12: Player performance in close matches (without matches won)


,full_name,avg_runs_in_close,close_matches_played
0,Rohit Sharma,80.0,1
1,Virat Kohli,95.0,1
2,Pat Cummins,60.0,1
3,Glenn Maxwell,75.0,1
4,Jos Buttler,65.0,1


In [54]:
query13 = """
SELECT p.full_name, 
       CAST(s.economy_rate AS FLOAT) AS economy
FROM player_format_summary s
JOIN players p ON s.player_id = p.player_id
WHERE s.economy_rate IS NOT NULL
ORDER BY economy ASC
LIMIT 5;
"""
conn = sqlite3.connect("cricbuzz.db")
df13 = pd.read_sql(query13, conn)
print("Query 13: Top 5 bowlers with best economy rate")
display(df13)


Query 13: Top 5 bowlers with best economy rate


,full_name,economy
0,Jasprit Bumrah,4.6
1,Pat Cummins,5.0
2,Glenn Maxwell,5.6


In [55]:
query14 = """
SELECT p.full_name, 
       CAST(s.strike_rate AS FLOAT) AS strike_rate
FROM player_format_summary s
JOIN players p ON s.player_id = p.player_id
WHERE s.strike_rate IS NOT NULL
ORDER BY strike_rate DESC
LIMIT 5;
"""
df14 = pd.read_sql(query14, conn)
print("Query 13: Top 5 batsmen with highest strike rate")
display(df14)

Query 13: Top 5 batsmen with highest strike rate


,full_name,strike_rate
0,Glenn Maxwell,124.4
1,Jos Buttler,110.2
2,Virat Kohli,93.4
3,Rohit Sharma,92.1


In [56]:
# Query 15 – Top 10 batsmen by batting average
query15 = """
SELECT p.full_name, 
       CAST(s.batting_average AS FLOAT) AS batting_average
FROM player_format_summary s
JOIN players p ON s.player_id = p.player_id
WHERE s.batting_average IS NOT NULL
ORDER BY batting_average DESC
LIMIT 10;
"""
df15 = pd.read_sql(query15, conn)
print("Query 15: Top 10 batsmen by batting average")
display(df15)


Query 15: Top 10 batsmen by batting average


,full_name,batting_average
0,Virat Kohli,57.7
1,Rohit Sharma,48.5
2,Jos Buttler,41.0
3,Glenn Maxwell,35.8


In [57]:
# Query 16 – Top 10 bowlers by bowling average
query16 = """
SELECT p.full_name, 
       CAST(s.bowling_average AS FLOAT) AS bowling_average
FROM player_format_summary s
JOIN players p ON s.player_id = p.player_id
WHERE s.bowling_average IS NOT NULL
ORDER BY bowling_average ASC
LIMIT 10;
"""
df16 = pd.read_sql(query16, conn)
print("Query 16: Top 10 bowlers by bowling average")
display(df16)

Query 16: Top 10 bowlers by bowling average


,full_name,bowling_average
0,Jasprit Bumrah,24.1
1,Pat Cummins,26.2
2,Glenn Maxwell,38.5


In [58]:
# Query 17 – Players with most matches played
query17 = """
SELECT p.full_name, 
       s.matches
FROM player_format_summary s
JOIN players p ON s.player_id = p.player_id
WHERE s.matches IS NOT NULL
ORDER BY s.matches DESC
LIMIT 10;
"""
df17 = pd.read_sql(query17, conn)
print("Query 17: Players with most matches played")
display(df17)

Query 17: Players with most matches played


,full_name,matches
0,Virat Kohli,289
1,Rohit Sharma,254
2,Jos Buttler,180
3,Glenn Maxwell,150
4,Pat Cummins,120
5,Jasprit Bumrah,85


In [59]:
# Query 18 – Best all-rounders (runs + wickets)
query18 = """
SELECT p.full_name, 
       s.runs, 
       s.wickets,
       (s.runs + s.wickets * 25) AS allrounder_points
FROM player_format_summary s
JOIN players p ON s.player_id = p.player_id
ORDER BY allrounder_points DESC
LIMIT 10;
"""
df18 = pd.read_sql(query18, conn)
print("Query 19: Best all-rounders (runs + wickets)")
display(df18)
conn.close()

Query 19: Best all-rounders (runs + wickets)


,full_name,runs,wickets,allrounder_points
0,Virat Kohli,13800,4,13900
1,Rohit Sharma,10700,8,10900
2,Pat Cummins,450,190,5200
3,Jos Buttler,5200,0,5200
4,Glenn Maxwell,3800,50,5050
5,Jasprit Bumrah,60,150,3810
